In [1]:
import cv2
import mediapipe as mp

In [8]:
# MediaPipe의 얼굴 감지, 그리기 유틸리티, 포즈 감지 모듈을 불러옵니다.
mp_face_detection = mp.solutions.face_detection
"""
mp.solutions.drawing_utils:
이 모듈은 랜드마크(landmarks)를 이미지 위에 그리는 데 사용되는 유틸리티 함수들을 제공합니다.
예를 들어, 감지된 얼굴이나 포즈의 랜드마크를 이미지 위에 시각적으로 표시하고 싶을 때 draw_landmarks 함수를 사용할 수 있습니다.

mp.solutions.drawing_styles:
이 모듈은 drawing_utils에서 랜드마크를 그릴 때 사용할 수 있는 스타일과 색상 설정을 정의합니다.
예를 들어, 랜드마크나 연결선의 색상, 두께 등을 지정할 때 사용할 수 있는 스타일 객체를 제공합니다.

간단히 말하자면, drawing_utils는 그리기 기능 자체를 담당하며, drawing_styles는 그리는 방식을 꾸며주는 스타일을 정의하는 데 사용됩니다.
이를 통해 개발자는 랜드마크를 그릴 때 일관되고 커스텀 가능한 스타일을 적용할 수 있습니다."""
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# 웹캠을 사용하기 위해 VideoCapture 객체를 초기화합니다.
cap = cv2.VideoCapture(0)
# 얼굴 감지 및 포즈 객체를 초기화합니다.
"""백슬래시(\)는 파이썬에서 라인을 연속으로 이어주는 문자"""
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection,\
    mp_pose.Pose(
      min_detection_confidence = 0.5,
      min_tracking_confidence = 0.5) as pose:
  # 웹캠이 열려있는 동안 반복합니다.
  while cap.isOpened():
    # 웹캠으로부터 프레임을 읽어옵니다.
    success, image = cap.read()
    # 프레임을 읽지 못했으면 경고를 출력하고 다음 프레임으로 넘어갑니다.
    if not success:
      print("빈 카메라 프레임을 무시합니다.")
      continue

    # 성능 향상을 위해 이미지를 수정 불가로 설정합니다.
    image.flags.writeable = False
    # 이미지를 BGR에서 RGB로 변환합니다.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 얼굴 감지를 수행합니다.
    result1 = face_detection.process(image)
    # 포즈 감지를 수행합니다.
    result2 = pose.process(image)

    # 이미지를 다시 쓰기 가능하게 하고 BGR로 다시 변환합니다.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # 포즈 랜드마크를 그립니다.
    if result2.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            result2.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    # 감지된 얼굴에 대한 주석을 그립니다.
    if result1.detections:
      for detection in result1.detections:
        mp_drawing.draw_detection(image, detection)
    
    # 이미지를 수평으로 뒤집어서 자기 자신을 보는 것처럼 표시합니다.
    cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
    # ESC 키를 누르면 반복을 종료합니다.
    if cv2.waitKey(5) & 0xFF == 27:
      break

# VideoCapture 객체를 해제합니다.
cap.release()

KeyboardInterrupt: 